In [2]:
import mysql.connector as my  # Import the MySQL connector
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# Fetch database credentials from environment variables
db_host = os.getenv("DB_HOST")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_name = os.getenv("DB_NAME")

connection = my.connect(
    host=db_host,
    user=db_user,
    password=db_password,
    database=db_name
)

In [3]:
# Create a cursor object to execute SQL queries
cursor = connection.cursor()

In [4]:
import pandas as pd
# Query data from the 'customer' table
cursor.execute('SELECT * FROM customer')
 #After fetching data from the database we are storing it into Pandas DataFrame
customer_data = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in
 cursor.description])
 # Query data from the 'product' table
cursor.execute('SELECT * FROM product')
product_data = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in
cursor.description])
 # Query data from the 'order_details' table
cursor.execute('SELECT * FROM orders')
order_data = pd.DataFrame(cursor.fetchall(), columns=[desc[0] for desc in
cursor.description])
 #printing first 5 records from each table

In [5]:
print(customer_data.head())

  customer_id        name     city                  email    phone_no  \
0    CUST0001  Customer 1  Chicago  customer1@example.com  1785014047   
1    CUST0002  Customer 2  Phoenix  customer2@example.com  2882940227   
2    CUST0003  Customer 3  Houston  customer3@example.com  4062157789   
3    CUST0004  Customer 4  Chicago  customer4@example.com  8729546243   
4    CUST0005  Customer 5  Chicago  customer5@example.com  8135387695   

       address  pin_code  
0  996 Main St     20021  
1  441 Main St     53492  
2  187 Main St     30487  
3  156 Main St     76466  
4  591 Main St     23771  


In [6]:
print(product_data.head())

  product_id product_name        category sub_category  original_price  \
0   PROD0001    Product 1  Home & Kitchen   Appliances          915.81   
1   PROD0002    Product 2          Sports   Appliances          851.54   
2   PROD0003    Product 3           Books   Appliances          454.96   
3   PROD0004    Product 4          Sports    Equipment          104.46   
4   PROD0005    Product 5          Sports      Fiction          377.11   

   selling_price  stock  
0         888.64     10  
1         826.78     91  
2         420.03     56  
3          90.26     88  
4         364.15     49  


In [7]:
print(order_data.head())

   order_id customer_id product_id  quantity total_price      payment_mode  \
0         1    CUST0052   PROD0009         1      562.53       Credit Card   
1         2    CUST0093   PROD0037         8      789.12            PayPal   
2         3    CUST0015   PROD0033         1      867.09       Credit Card   
3         4    CUST0072   PROD0042         9     3485.25            PayPal   
4         5    CUST0061   PROD0044         6     3469.98  Cash on Delivery   

   order_date order_status  
0  2022-01-01      Pending  
1  2022-01-02      Pending  
2  2022-01-03      Shipped  
3  2022-01-04      Shipped  
4  2022-01-05    Cancelled  


Data Cleaning:

In [13]:
# Checkformissing values in each table.
print("\n Missing value in the customer data ")
print(customer_data.isnull().sum())
print("\n Missing value in the product data ")
print(product_data.isnull().sum())
print("\n Missing value in the order data ")
print(order_data.isnull().sum())


 Missing value in the customer data 
customer_id    0
name           0
city           0
email          0
phone_no       0
address        0
pin_code       0
dtype: int64

 Missing value in the product data 
product_id        0
product_name      0
category          0
sub_category      0
original_price    0
selling_price     0
stock             0
dtype: int64

 Missing value in the order data 
order_id        0
customer_id     0
product_id      0
quantity        0
total_price     0
payment_mode    0
order_date      0
order_status    0
dtype: int64


In [14]:
# 2. Ensuring Appropriate Data Types
print("\nData types in Customers Data:")
print(customer_data.dtypes)
print("\nData types in product Data:")
print(product_data.dtypes)
print("\nData types in order Data:")
print(order_data.dtypes)


Data types in Customers Data:
customer_id    object
name           object
city           object
email          object
phone_no       object
address        object
pin_code        int64
dtype: object

Data types in product Data:
product_id         object
product_name       object
category           object
sub_category       object
original_price    float64
selling_price     float64
stock               int64
dtype: object

Data types in order Data:
order_id         int64
customer_id     object
product_id      object
quantity         int64
total_price     object
payment_mode    object
order_date      object
order_status    object
dtype: object
